# 開啓這個 Colab Notebook 之後

請選擇 Save a Copy to Drive，將這個 Notebook 複製到你的 Google Drive 内

![alt text](https://drive.google.com/uc?export=download&id=1Lm7c12TZfS509ayfUO8lyacKCYD_C5hy)

# Python 與 Google Sheet 的整合術

在前面兩個教學，我們學會了可以透過 gspread 套件操作 Google Spreadsheet

但是，gspread 有一個嚴重的缺點，那就是**功能相對不齊全**，舉例來説，像是讀寫 Dataframe 的功能，gspread 套件本身並不具備，而是需要另外再安裝 gspread dataframe 才行。

在這篇教學，我們來探索另外一個串接 Google Spreadsheet 的 Python 套件：pygsheet

In [0]:
!pip install --upgrade --quiet pygsheets

     |████████████████████████████████| 102kB 2.2MB/s 


In [0]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [0]:
# 建立一個名爲 ”我的 Google Sheet“ 的 Google Spreadsheet
gc.create("pygsheet 測試")

<Spreadsheet 'pygsheet 測試' Sheets:1>

## 檢查一下是否有一個 Google Sheet 被新增出來

Google Sheet： [連結](https://docs.google.com/spreadsheets/u/0/)

## 請複製範例 Google Sheet

範例 Google Sheet：[連結](https://docs.google.com/spreadsheets/d/1aCLxFmDKCWzcjHSmAEEqW_lvgw_O_LKdi8ytK-JjJmo/edit?usp=sharing)

In [0]:
wb = gc.open("Copy of Py x Google Sheet： pygsheet 練習")
wb

In [0]:
wb.id

In [0]:
wb.title

In [0]:
wb.url

## 選擇工作表


In [0]:
wks = wb.worksheet_by_title("練習1")
wks

<Worksheet '練習1' index:0>

## 選擇單一儲存格

可以直接使用 `worksheet.cell()` 選擇儲存格

與我們之前學的 `xlwings` 内的 `cells()` 物件很類似   

In [0]:
# 選擇 A1 儲存格
wks.cell("A1")

In [0]:
# 讀取 A1 儲存格的值
wks.cell("A1").value

In [0]:
# 將 "Hello World!" 寫入 A2
wks.cell("A2").value = "Hello World!"

In [0]:
# 若畫面沒有顯示 'Hello World'，可以使用 refresh() 即時讓工作表更新
wks.refresh()

## 選擇儲存格範圍

可以使用 `worksheet.range()` 選擇儲存格範圍



In [0]:
# 會將資料以巢狀串列回傳
wks.range("A7:B7")
# [[<Cell A7 ''>, <Cell B7 ''>]]

In [0]:
# 預設上 range ，都會以 cells 物件的形式回傳
wks.range("A7:B7", returnas="cells")

In [0]:
# 當然，也可以將 returnas 參數設定成 matrix，回傳 cell 内的值
wks.range("A1:A2", returnas="matrix")

In [0]:
# 若要更新一個 range 的資料，可以使用 update_values
wks.update_values("A7:B7", values=[["ABC", "ABC"]])

In [0]:
# update_values 也有類似 xlwings 的功能：指定欲寫入範圍的起點即可
wks.update_values("A7", values=[["ABC", "ABC"], ["ABC", "ABC"]])

In [0]:
wks.cell("C12").value = wks.cell("A12").value
wks.refresh()

In [0]:
data = wks.range("A16:B18", returnas="matrix")

wks.update_values("D16", values=data)

In [0]:
# pygsheet 也具備了 formula 屬性，允許使用者讀取或是設定儲存格公式
wks.cell("C17").formula = "=sum(A17:A19)"

In [0]:
# pygsheet 也有設定顔色的功能，唯一要注意的是 RGB 值是 0 到 1 之間，與 xlwings 的 0 到 255 不一樣
wks.cell("A24").color = (1, 0, 0, 0)

In [0]:
wks2 = wb.worksheet_by_title("銷售數據")
wks2

In [0]:
df = wks2.get_as_df(has_header=True, index_column="日期", start="A1")
df

In [0]:
df = df.set_index("日期")
df

In [0]:
# 產生樞紐報表，但是結果似乎出乎我們意料...
df.groupby("產品").sum()

In [0]:
# 原因是，現在我們的金額資料是以字串的形式被解讀，我們需要將金額的資料轉成數字
info()

In [0]:
# 用 map + lambda 把金額的資料改變成浮點數
df["金額"] = df["金額"].map(lambda num_str : float(num_str.replace(",", "")))
df

In [0]:
# 產生樞紐報表
df2 = df.groupby("產品").sum().sort_values("金額", ascending=False)
df2

In [0]:
# 截取報告試算表
report_wks = wb.worksheet_by_title("報告")
report_wks

<Worksheet '報告' index:3>

In [0]:
# 設定 DataFrame
report_wks.set_dataframe(df2, copy_index=True, start="A1")

pygsheet 官方文件：[連結](https://pygsheets.readthedocs.io/en/stable/index.html)